# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [3]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [4]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [84]:
df1=data.groupby(by=["CustomerID","ProductName"]).agg({"Quantity":"sum"}).reset_index()
df1

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
...,...,...,...
63623,98200,Vol Au Vents,50
63624,98200,Wasabi Powder,25
63625,98200,Wine - Fume Blanc Fetzer,25
63626,98200,Wine - Hardys Bankside Shiraz,25


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [85]:
df2=pd.pivot_table(df1,index="ProductName",columns="CustomerID",values="Quantity").fillna(0)
df2.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [86]:
from scipy.spatial.distance import pdist, squareform
distances = pd.DataFrame(1/(1 + squareform(pdist(df2.T, 'euclidean'))), 
                         index=df2.columns, columns=df2.columns)
distances.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [87]:
listapaver=list(distances[264].sort_values(ascending=False)[1:6].index)
listapaver

[1008, 2617, 1072, 33, 3535]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [96]:
df5=df1[df1["CustomerID"].isin(listapaver)]
df5.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [101]:
df6=df5.groupby(by=["ProductName"]).agg({"Quantity": "sum"}).sort_values(by="Quantity",ascending=False).reset_index()
df6.head()

,ProductName,Quantity
0,Soupfoamcont12oz 112con,3
1,Wine - Two Oceans Cabernet,3
2,Bread - Italian Roll With Herbs,3
3,"Veal - Inside, Choice",3
4,Potatoes - Idaho 100 Count,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [144]:
boughtandnot=df2[264].to_frame()
bought=products264[products264[264]>0.0]
print(len(bought))

62


In [142]:
products264=df2[264].to_frame()
df7=pd.merge(products264,df6,on="ProductName")
df8=df7[df7[264]==0].sort_values(by="Quantity",ascending=False)
df8.head()

,ProductName,264,Quantity
177,Soupfoamcont12oz 112con,0.0,3
218,Wine - Two Oceans Cabernet,0.0,3
199,"Veal - Inside, Choice",0.0,3
36,Bread - Italian Roll With Herbs,0.0,3
84,Fondant - Icing,0.0,3


In [149]:
bought1 = list(data[data['CustomerID'] == 264]['ProductName'])
print(len(bought1))
df9=df6[~df6["ProductName"].isin(bought1)]
df9.head()
#no es exactamente igual... No sé qué diablos

65


,ProductName,Quantity
0,Soupfoamcont12oz 112con,3
1,Wine - Two Oceans Cabernet,3
2,Bread - Italian Roll With Herbs,3
3,"Veal - Inside, Choice",3
7,"Fish - Scallops, Cold Smoked",3


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [169]:
dicti={}
listcon=list(distances.index)
for con in listcon:
    listapaver=list(distances[con].sort_values(ascending=False)[1:6].index)
    df10=df1[df1["CustomerID"].isin(listapaver)]
    df11=df10.groupby(by=["ProductName"]).agg({"Quantity": "sum"}).sort_values(by="Quantity",ascending=False).reset_index()
    boughtcon = list(data[data['CustomerID'] == con]['ProductName'])
    df12=df11[~df11["ProductName"].isin(boughtcon)].head()
    dicti[con]=list(df12["ProductName"])
dicti[33]    

['Butter - Unsalted',
 'Wine - Ej Gallo Sierra Valley',
 'Soup - Campbells Bean Medley',
 'Wine - Blue Nun Qualitatswein',
 'Chicken - Soup Base']

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [171]:
df13=pd.DataFrame(dicti)
df13.T.head()

,0,1,2,3,4
33,Butter - Unsalted,Wine - Ej Gallo Sierra Valley,Soup - Campbells Bean Medley,Wine - Blue Nun Qualitatswein,Chicken - Soup Base
200,Soup - Campbells Bean Medley,Muffin - Carrot Individual Wrap,Bay Leaf,Pork - Kidney,Wanton Wrap
264,Soupfoamcont12oz 112con,Wine - Two Oceans Cabernet,Bread - Italian Roll With Herbs,"Veal - Inside, Choice","Fish - Scallops, Cold Smoked"
356,"Veal - Inside, Choice",Lamb - Ground,Pomello,Butter - Unsalted,"Tart Shells - Sweet, 4"
412,Olive - Spread Tapenade,Sprouts - Baby Pea Tendrils,Wine - Blue Nun Qualitatswein,"Pepper - Black, Whole",Soup - Campbells Bean Medley


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [174]:
distances = pd.DataFrame(squareform(pdist(df2.T, 'chebyshev')), 
                         index=df2.columns, columns=df2.columns)
distances.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.0,3.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,...,25.0,50.0,50.0,50.0,75.0,50.0,50.0,50.0,50.0,50.0
200,3.0,0.0,3.0,3.0,2.0,3.0,2.0,3.0,3.0,3.0,...,25.0,50.0,50.0,50.0,75.0,50.0,49.0,50.0,50.0,50.0
264,2.0,3.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,25.0,50.0,50.0,50.0,75.0,50.0,50.0,50.0,50.0,50.0
356,2.0,3.0,2.0,0.0,2.0,2.0,3.0,2.0,2.0,2.0,...,25.0,50.0,50.0,50.0,75.0,50.0,50.0,50.0,50.0,50.0
412,2.0,2.0,2.0,2.0,0.0,2.0,3.0,2.0,2.0,2.0,...,25.0,50.0,50.0,50.0,74.0,50.0,50.0,50.0,50.0,50.0
